## Byte-at-a-time ECB decryption (Simple)
 Copy your oracle function to a new function that encrypts buffers under ECB mode using a _consistent_ but _unknown_ key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string: 

```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```

|Spoiler alert.|
| ------------ |
|Do not decode this string now. Don't do it.|

Base64 decode the string before appending it. _Do not base64 decode the string by hand; make your code do it._ The point is that you don't know its contents.

What you have now is a function that produces: 

`AES-128-ECB(your-string || unknown-string, random-key)`

It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how: 

1. Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
2. Detect that the function is using ECB. You already know, but do this step anyways.
2. Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
2. Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
2. Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
2. Repeat for the next byte.


|Congratulations.|
| -------------- |
|This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.|


In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from os import urandom
from base64 import b64decode

In [2]:
KEY = urandom(16) # 128-bit AES key

In [3]:
FLAG_b64 = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"

In [4]:
FLAG = b64decode(FLAG_b64)

In [5]:
def encryption_oracle(plaintext:bytes):
    plaintext = plaintext + FLAG
    padded = pad(plaintext, AES.block_size)
    cipher = AES.new(KEY, AES.MODE_ECB)
    return cipher.encrypt(padded)


### Confirm block size
Passing a byte string of length $1$ to the oracle function returns a ciphertext of length $144$.

Consider $144 = len(FLAG) + 1 + 16$ where $1$ is the length of the input and $16$ is the expected length of padding if the unknown string length is $127$.

In [9]:
ciphertext = encryption_oracle(b'0')
len(ciphertext)

144

Pass a byte string of length $2$ to the oracle function.

$144 = len(FLAG) + 2 + 15$, which confirms AES block size (which we know) and length of unknown string.

In [10]:
ciphertext = encryption_oracle(b'00')
len(ciphertext)

144